<a href="https://colab.research.google.com/github/Rishujamaiyar/desktop-tutorial/blob/master/Copy_of_pilot_KG_Embedd_entities_allfeat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.0.0rc1

  # verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.0.0rc1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.0.0rc1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

     |████████████████████████████████| 378kB 2.6MB/s 
     |████████████████████████████████| 235kB 13.1MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 


In [74]:
import pandas as pd
import numpy as np
import random

from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UniformRandomWalk
from stellargraph.data import UnsupervisedSampler

In [75]:
nodelink=pd.read_csv('/content/ckt_links.csv',names=['source','target'])
nodelink=nodelink.drop(nodelink.index[0])
nodelink

,source,target
1,Kedar Jadhav,Bat_IND
2,Mayank Agarwal,Bat_IND
3,Rohit Sharma,Bat_IND
4,Shikhar Dhawan,Bat_IND
5,Virat Kohli,Bat_IND
...,...,...
196,SL,WC
197,SA,WC
198,BAN,WC
199,WI,WC


In [76]:
nodefeat=pd.read_csv('/content/ckt_feat.csv')
nodefeat=nodefeat.set_index('players')
nodefeat_raw = nodefeat
nodefeat=pd.get_dummies(nodefeat,columns=['feat'])
nodefeat

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,...,feat_162,feat_163,feat_164,feat_165,feat_166,feat_167,feat_168,feat_169,feat_170,feat_171,feat_172,feat_173,feat_174,feat_175,feat_176,feat_177,feat_178,feat_179,feat_180,feat_181,feat_182,feat_183,feat_184,feat_185,feat_186,feat_187,feat_188,feat_189,feat_190,feat_191,feat_192,feat_193,feat_194,feat_195,feat_196,feat_197,feat_198,feat_199,feat_200,feat_201
players,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Kedar Jadhav,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Mayank Agarwal,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Rohit Sharma,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Shikhar Dhawan,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Virat Kohli,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
BAN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [78]:
from stellargraph import StellarGraph
datagraph = StellarGraph( {"source": nodefeat}, {"line": nodelink})
print(datagraph.info())

StellarGraph: Undirected multigraph
 Nodes: 201, Edges: 200

 Node types:
  source: [201]
    Features: float32 vector, length 201
    Edge types: source-line->source

 Edge types:
    source-line->source: [200]
        Weights: all 1 (default)


In [79]:
#Unsupervised GraphSAGE implementation starts here

nodes = list(datagraph.nodes())
number_of_walks = 5
length = 4
batch_size = 10
epochs = 5
num_samples = [5, 4]

In [80]:
unsupervised_samples = UnsupervisedSampler(datagraph, nodes=nodes, length=length, number_of_walks=number_of_walks)

In [81]:
generator = GraphSAGELinkGenerator(datagraph, batch_size, num_samples)
train_gen = generator.flow(unsupervised_samples)

In [82]:
layer_sizes = [40, 40]            #determine the size of layers and hence output vectors
graphsage = GraphSAGE(layer_sizes=layer_sizes, generator=generator, bias=True, dropout=0.0, normalize="l2")

In [83]:
# Build the model and expose input and output sockets of graphsage, for node pair inputs:
x_inp, x_out = graphsage.in_out_tensors()
prediction = link_classification(output_dim=1, output_act="sigmoid", edge_embedding_method="ip")(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [84]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

In [85]:
history = model.fit(
    train_gen,
    epochs=epochs,
    verbose=1,
    use_multiprocessing=False,
    workers=4,
    shuffle=True,
)

Epoch 1/5
603/603 [==============================] - 6s 10ms/step - loss: 0.5834 - binary_accuracy: 0.7166
Epoch 2/5
603/603 [==============================] - 6s 10ms/step - loss: 0.5397 - binary_accuracy: 0.7474
Epoch 3/5
603/603 [==============================] - 6s 10ms/step - loss: 0.5363 - binary_accuracy: 0.7570
Epoch 4/5
603/603 [==============================] - 6s 10ms/step - loss: 0.5251 - binary_accuracy: 0.7794
Epoch 5/5
603/603 [==============================] - 6s 10ms/step - loss: 0.5248 - binary_accuracy: 0.7867


In [86]:
x_inp_src = x_inp[0::2]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

In [87]:
node_ids = nodefeat.index
node_gen = GraphSAGENodeGenerator(datagraph, batch_size, num_samples).flow(node_ids)

In [88]:
node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)

21/21 [==============================] - 0s 7ms/step


In [89]:
node_embedd=pd.DataFrame(node_embeddings)
node_embedd['nodes']=nodes
node_embedd=node_embedd.set_index('nodes')
node_embedd

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
nodes,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Kedar Jadhav,0.292280,0.056647,-0.262089,0.003980,-0.076056,0.052593,-0.055122,-0.101379,-0.206529,-0.064279,-0.017356,0.097088,0.159895,0.037582,-0.065818,0.160330,0.010954,-0.173763,-0.079958,0.097588,-0.160301,-0.018526,-0.187540,-0.021803,-0.101526,-0.184155,0.158137,0.390427,0.158838,-0.009203,-0.201514,0.154706,-0.022875,0.114511,0.109329,0.313659,0.276787,-0.278417,-0.054648,0.082107
Mayank Agarwal,0.272155,0.099263,-0.243889,0.013179,-0.127486,0.029820,-0.013500,-0.106994,-0.182825,-0.045790,0.028234,0.082182,0.203127,0.062685,-0.092170,0.102560,0.000609,-0.191952,0.003946,0.070629,-0.153807,-0.030444,-0.189957,-0.003821,-0.121522,-0.182517,0.142391,0.388737,0.159589,0.004919,-0.243273,0.136858,-0.022217,0.119198,0.102117,0.303017,0.283181,-0.286260,-0.052138,0.114731
Rohit Sharma,0.316569,0.044673,-0.267582,0.032701,-0.063212,0.085152,-0.047418,-0.094766,-0.221316,-0.048173,-0.040861,0.097376,0.160200,0.051717,-0.070390,0.147122,0.043633,-0.212105,-0.075532,0.088612,-0.152574,-0.034476,-0.175623,-0.012666,-0.110974,-0.177349,0.135329,0.364753,0.168255,-0.004148,-0.230888,0.133306,-0.009844,0.100340,0.099278,0.309573,0.268754,-0.271688,-0.040246,0.106200
Shikhar Dhawan,0.269974,0.045036,-0.268864,0.014538,-0.083343,0.073109,-0.056096,-0.098112,-0.200035,-0.046676,-0.021854,0.082994,0.136230,0.056643,-0.083025,0.154360,0.067142,-0.205017,-0.103812,0.110550,-0.150447,-0.028549,-0.191436,-0.003748,-0.115073,-0.180498,0.143140,0.365969,0.170043,-0.001188,-0.238305,0.149632,-0.013037,0.097904,0.099155,0.299887,0.284178,-0.288133,-0.051525,0.107410
Virat Kohli,0.305277,0.046292,-0.283788,0.004528,-0.087205,0.042213,-0.022424,-0.077621,-0.202520,-0.050006,0.000516,0.069855,0.207337,0.048577,-0.078940,0.099965,0.027705,-0.217809,-0.037599,0.123332,-0.173918,-0.040165,-0.191621,-0.034143,-0.071609,-0.151755,0.139150,0.361397,0.177463,0.004297,-0.250108,0.143981,0.017548,0.083265,0.089759,0.306650,0.270497,-0.265505,-0.044504,0.125493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SA,-0.084763,-0.113558,-0.083624,0.009571,-0.030005,0.097219,0.089951,-0.080451,0.135462,0.132102,-0.047145,-0.164905,-0.095662,0.152843,0.021036,-0.163975,0.170557,0.215077,-0.094512,0.208128,0.084350,0.073653,-0.007869,-0.099774,0.208735,-0.090742,-0.100339,-0.243725,-0.130537,-0.330090,0.275832,-0.244942,0.226957,0.163737,0.135433,0.035577,-0.180913,0.147233,-0.172231,-0.327650
BAN,-0.048526,0.003726,-0.046989,-0.072583,-0.229890,-0.051829,-0.186431,0.005577,-0.227811,-0.093958,0.157381,0.199125,0.103656,-0.112644,0.018395,0.134007,0.107792,-0.257777,0.151081,-0.152100,0.164346,0.258414,-0.012498,0.061118,-0.179063,-0.186134,0.221206,0.333459,-0.233890,0.142896,-0.064313,0.088227,-0.296398,0.056088,0.079036,0.042217,0.102526,-0.221315,-0.108310,-0.164680
WI,0.085855,-0.026329,0.188422,0.208438,0.272377,-0.073486,0.254118,0.188227,0.189222,0.166843,-0.016172,0.068667,-0.021109,0.048562,0.160558,0.095011,0.158244,0.168777,0.326352,0.070765,0.236266,-0.173682,0.059607,-0.020287,-0.088031,-0.083667,-0.222770,-0.197498,-0.101200,0.085178,-0.205141,-0.099795,0.141622,-0.227501,-0.160229,-0.205250,0.046487,0.017665,0.237532,-0.027263


In [92]:
from sklearn.metrics.pairwise import cosine_similarity
a = 'Virat Kohli'
b = ['Mayank Agarwal','Yuzvendra Chahal','David Warner','Soumya Sarkar','Faf du Plessis']

for each in b:
  x=node_embedd.loc[[a]]
  y=node_embedd.loc[[each]]
  print(abs(cosine_similarity(x,y)),a,"<----->" ,each)

[[0.98712146]] Virat Kohli <-----> Mayank Agarwal
[[0.81957287]] Virat Kohli <-----> Yuzvendra Chahal
[[0.32843906]] Virat Kohli <-----> David Warner
[[0.21252364]] Virat Kohli <-----> Soumya Sarkar
[[0.37364566]] Virat Kohli <-----> Faf du Plessis
